<a href="https://colab.research.google.com/github/kpriyanka93/E2E_travel_app_project/blob/main/app_flight_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**REST API for Regression Model**

In [1]:
!pip install --upgrade scikit-learn==1.3.2

In [2]:
!pip install flask-ngrok

In [3]:
!pip install Flask pyngrok


In [4]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import os
import pickle

In [6]:
import glob
import joblib
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pyngrok import ngrok
from datetime import datetime
from google.colab import drive
from flask_ngrok import run_with_ngrok
from sklearn.preprocessing import StandardScaler
from flask import Flask, render_template, request, jsonify

In [7]:
app = Flask(__name__)


In [8]:
!ngrok authtoken "39Qq742oSO2Jr00Ku5epsQccet9_6qoruv5ywAN6nU689ETpD"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
'''
# Open the file in read-binary ('rb') mode
with open('/content/drive/MyDrive/flight_price_main.pkl', 'rb') as file:
    model = pickle.load(file)
'''

UnpicklingError: invalid load key, '\x08'.

In [13]:
import joblib

model = joblib.load('/content/drive/MyDrive/flight_price_main.pkl')


In [14]:
# Open the file in read-binary ('rb') mode
with open('/content/drive/MyDrive/label.pkl', 'rb') as file:
    label = pickle.load(file)

In [15]:
with open('/content/drive/MyDrive/label_encoders.pkl', 'rb') as file:
    label_encoders = pickle.load(file)

In [16]:
# Load the scaler
scaler = joblib.load('/content/drive/MyDrive/std_regressor.pkl')

# Check the type to ensure it's a StandardScaler
print(type(scaler))

<class 'sklearn.preprocessing._data.StandardScaler'>


In [17]:
with open('/content/drive/MyDrive/FEATURE_COLUMNS.pkl', 'rb') as file:
    FEATURE_COLUMNS = pickle.load

In [18]:
def predict_price(input_data, model, label_encoders, scaler):
    CAT_COLS = ['from', 'to', 'flightType', 'agency']
    NUM_COLS = ['month', 'year']

    # Create DataFrame
    df = pd.DataFrame([input_data])

    # Encode categorical columns
    for col in CAT_COLS:
        le = label_encoders[col]
        df[col] = df[col].astype(str).apply(
            lambda x: le.transform([x])[0] if x in le.classes_ else -1
        )

    # Convert numericals
    df[NUM_COLS] = df[NUM_COLS].astype(float)
    FEATURE_COLUMNS=['from', 'to', 'flightType', 'agency', 'month', 'year']
    # Reorder exactly like training
    df = df[FEATURE_COLUMNS]

    # Scale → NumPy
    x_scaled = scaler.transform(df.values)

    # Predict using NumPy
    return model.predict(x_scaled)[0]


In [19]:
input_data = {
    "from": "Recife (PE)",
    "to": "Natal (RN)",
    "flightType": "economic",
    "agency": "CloudFy",
    "month": 1,
    "year": 2020
}

price = predict_price(
    input_data,
    model,
    label_encoders,
    scaler
)

print(price)


654.8800000000036


In [20]:
print(type(label_encoders))
print(label_encoders.keys())


<class 'dict'>
dict_keys(['from', 'to', 'flightType', 'agency', 'date', 'route'])


In [21]:
!pip install -q pyngrok flask


In [22]:
from flask import Flask
from pyngrok import ngrok


In [23]:
ngrok.set_auth_token("39Qq742oSO2Jr00Ku5epsQccet9_6qoruv5ywAN6nU689ETpD")


In [ ]:
NGROK_AUTH_TOKEN = "39Qq742oSO2Jr00Ku5epsQccet9_6qoruv5ywAN6nU689ETpD"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

app = Flask(__name__)
#run_with_ngrok(app)


@app.route('/', methods=['GET', 'POST'])
def predict():
    return """

<!DOCTYPE html>
<html>

<head>
    <title>Flight Price Prediction</title>
    <style>
        body {
            font-family: 'Poppins', sans-serif;
            background-color: #f9f9f9;
            margin: 0;
            padding: 0;
        }

        .container {
            max-width: 800px;
            margin: 0 auto;
            padding: 40px;
            background-color: #ffffff;
            border-radius: 10px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.1);
            text-align: center;
        }

        h1 {
            color: #39dde0;
            font-size: 36px;
            margin-bottom: 20px;
        }

        form {
            text-align: left;
        }

        input[type="text"],
        input[type="number"] {
            width: 100%;
            padding: 15px;
            margin: 15px 0;
            border: none;
            border-bottom: 2px solid #39dde0;
            font-size: 18px;
            background-color: transparent;
            color: #39dde0;
            transition: border-bottom 0.3s ease;
        }

        input[type="text"]:focus,
        input[type="number"]:focus {
            border-bottom: 2px solid #39dde0;
            outline: none;
        }

        input[type="radio"] {
            margin-right: 10px;
        }

        input[type="submit"] {
            background-color: #39dde0;
            color: #fff;
            padding: 15px 30px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 20px;
            transition: background-color 0.3s ease;
        }

        input[type="submit"]:hover {
            background-color: #39dde0;
        }

        p#prediction {
            margin-top: 20px;
            font-size: 24px;
            color: #39dde0;
        }
    </style>
</head>

<body>
    <div class="container">
        <h1>Flight Price Prediction</h1>
        <hr style="border: 1px solid #39dde0; width: 80%; margin: 20px auto;">
        <form action="/predict" method="POST">

            <label>Departure City:</label><br><br>


            <input type="radio" name="From" value="Aracaju (SE)">Aracaju (SE)<br>
            <input type="radio" name="From" value="Brasilia (DF)">Brasilia (DF)<br>
            <input type="radio" name="From" value="Campo Grande (MS)">Campo Grande (MS)<br>
            <input type="radio" name="From" value="Florianopolis (SC)">Florianopolis (SC)<br>
            <input type="radio" name="From" value="Natal (RN)">Natal (RN)<br>
            <input type="radio" name="From" value="Recife (PE)">Recife (PE)<br>
            <input type="radio" name="From" value="Rio de Janeiro (RJ)">Rio de Janeiro (RJ)<br>
            <input type="radio" name="From" value="Salvador (BH)">Salvador (BH)<br>
            <input type="radio" name="From" value="Sao Paulo (SP)">Sao Paulo (SP)<br><br>

            <hr style="border: 1px solid #39dde0; width: 100%; margin: 20px auto;">

            <label>Destination City:</label><br><br>

            <input type="radio" name="To" value="Aracaju (SE)">Aracaju (SE)<br>
            <input type="radio" name="To" value="Brasilia (DF)">Brasilia (DF)<br>
            <input type="radio" name="To" value="Campo Grande (MS)">Campo Grande (MS)<br>
            <input type="radio" name="To" value="Florianopolis (SC)">Florianopolis (SC)<br>
            <input type="radio" name="To" value="Natal (RN)">Natal (RN)<br>
            <input type="radio" name="To" value="Recife (PE)">Recife (PE)<br>
            <input type="radio" name="To" value="Rio de Janeiro (RJ)">Rio de Janeiro (RJ)<br>
            <input type="radio" name="To" value="Salvador (BH)">Salvador (BH)<br>
            <input type="radio" name="To" value="Sao Paulo (SP)">Sao Paulo (SP)<br><br>

            <hr style="border: 1px solid #39dde0; width: 100%; margin: 20px auto;">

            <label>Flight Type:</label><br><br>

            <input type="radio" name="flightType" value="economic"> Economic<br>
            <input type="radio" name="flightType" value="firstClass"> FirstClass<br>
            <input type="radio" name="flightType" value="premium"> Premium<br><br>

            <hr style="border: 1px solid #39dde0; width: 100%; margin: 20px auto;">


            <label>Agency:</label><br><br>


            <input type="radio" name="agency" value="CloudFy"> CloudFy<br>
            <input type="radio" name="agency" value="FlyingDrops"> FlyingDrops<br>
            <input type="radio" name="agency" value="Rainbow"> Rainbow<br><br>

            <hr style="border: 1px solid #39dde0; width: 100%; margin: 20px auto;">

            <label for="weekday_num">Weekday (0=Sunday, 6=Saturday):</label>
            <input type="number" name="weekday_num" min="0" max="6" placeholder="Day of the week"><br>

            <label for="month">Month:</label>
            <input type="number" name="month" min="1" max="12" placeholder="Month"><br>

            <label for="year">Year:</label>
            <input type="number" name="year" min="2019" max="2123" placeholder="Year"><br>


            <input type="submit" value="Predict">
        </form>
        <p id="prediction"></p>
    </div>
</body>

</html>


    """

@app.route('/predict', methods=['POST'])
def index():
    if request.method == 'POST':
        # Get input data from the form
        From = request.form.get('From')
        To = request.form.get('To')
        flighttype = request.form.get('flightType')
        agency = request.form.get('agency')
        weekday_num = request.form.get('weekday_num')
        month = request.form.get('month')
        year = request.form.get('year')

        # Create a dictionary to store the input data
        data = {
            'from': From,
            'to': To,
            'flightType': flighttype,
            'agency': agency,
            'weekday_num': weekday_num,
            'month': month,
            'year': year
        }

        # Perform prediction using the custom_input dictionary
        prediction = predict_price(data,model,label,scaler)
        prediction = str(prediction)

        return jsonify({'Your Flight Price($) will be around': prediction})

# Open a tunnel on the default port 5000
public_url = ngrok.connect(5000)
print(f"Your Flask app is running on {public_url}")

if __name__ == "__main__":
    app.run(port=5000)



Your Flask app is running on NgrokTunnel: "https://deedee-loganiaceous-purposively.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
